# Azure埋め込みの例

この例では、Azure OpenAIサービスを使用した埋め込みについて説明します。

## セットアップ

まず、必要な依存関係をインストールし、使用するライブラリをインポートします。

In [ ]:
! pip install "openai>=1.0.0,<2.0.0"
! pip install python-dotenv

In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()

### 認証

Azure OpenAI サービスは、APIキーとAzure Active Directoryトークン認証情報を含む複数の認証メカニズムをサポートしています。

In [2]:
use_azure_active_directory = False  # Set this flag to True if you are using Azure Active Directory

#### APIキーを使用した認証

OpenAI SDKを*Azure APIキー*を使用するように設定するには、`api_key`をエンドポイントに関連付けられたキーに設定する必要があります（このキーは[Azure Portal](https://portal.azure.com)の*「リソース管理」*の下にある*「キーとエンドポイント」*で確認できます）。ここでリソースのエンドポイントも確認できます。

In [3]:
if not use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=api_key,
        api_version="2023-09-01-preview"
    )

#### Azure Active Directoryを使用した認証
次に、Azure Active Directoryを介して認証する方法を見てみましょう。まず、`azure-identity`ライブラリをインストールします。このライブラリは、認証に必要なトークン認証情報を提供し、`get_bearer_token_provider`ヘルパー関数を通じてトークン認証情報プロバイダーを構築するのに役立ちます。`AzureOpenAI`に静的トークンを提供するよりも`get_bearer_token_provider`を使用することが推奨されます。なぜなら、このAPIは自動的にトークンをキャッシュし、更新してくれるからです。

Azure OpenAIでAzure Active Directory認証を設定する方法の詳細については、[ドキュメント](https://learn.microsoft.com/azure/ai-services/openai/how-to/managed-identity)を参照してください。

In [ ]:
! pip install "azure-identity>=1.15.0"

In [5]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

if use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        azure_ad_token_provider=get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),
        api_version="2023-09-01-preview"
    )

> 注意: AzureOpenAIは、以下の引数が提供されていない場合、対応する環境変数から推測します：

- `api_key` は `AZURE_OPENAI_API_KEY` から
- `azure_ad_token` は `AZURE_OPENAI_AD_TOKEN` から
- `api_version` は `OPENAI_API_VERSION` から
- `azure_endpoint` は `AZURE_OPENAI_ENDPOINT` から

## デプロイメント

このセクションでは、埋め込みを作成するために使用できるモデルのデプロイメントを作成します。

### デプロイメント: Azure OpenAI Studio での作成
埋め込みで使用するモデルをデプロイしましょう。https://portal.azure.com にアクセスし、Azure OpenAIリソースを見つけて、Azure OpenAI Studioに移動します。「Deployments」タブをクリックし、埋め込みに使用したいモデルのデプロイメントを作成します。モデルに付けるデプロイメント名は、以下のコードで使用されます。

In [4]:
deployment = "" # Fill in the deployment name from the portal here

## Embeddings

それでは、構築したクライアントを使用してembeddingsを作成してみましょう。

In [ ]:
embeddings = client.embeddings.create(
    model=deployment,
    input="The food was delicious and the waiter..."
)
                                
print(embeddings)